In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importing Libraries

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from keras import layers


In [ ]:
# Define the source directory where the images are located
source_dir = '/content/drive/MyDrive/RealWaste_dataset/RealWaste/'

# Define the destination directories for the training and testing sets
data_train_path = '/content/drive/MyDrive/RealWaste_dataset/train/'
data_test_path = '/content/drive/MyDrive/RealWaste_dataset/test/'


In [ ]:
img_width = 180
img_height =180

In [ ]:
data_train = keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32,
    validation_split=False)

NotFoundError: Could not find directory /content/drive/MyDrive/RealWaste_dataset/train/

In [ ]:
data_classes = data_train.class_names

In [ ]:
data_classes

In [ ]:
data_test = tf.keras.utils.image_dataset_from_directory(
data_test_path,
    image_size=(img_height,img_width),
    shuffle=False,
    batch_size=32,
    validation_split=False
)

In [ ]:
plt.figure(figsize=(10,10))
for image, labels in data_train.take(1):
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(data_classes[labels[i]])
        plt.axis('off')

In [ ]:
from keras.models import Sequential

data_train

In [ ]:
model = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,3, padding='same',activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(128),
    layers.Dense(len(data_classes))

])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs_size = 500
history = model.fit(data_train, validation_data=data_test, epochs=epochs_size)

In [ ]:
epochs_range = range(epochs_size)
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range,history.history['accuracy'],label = 'Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'],label = 'Validation Accuracy')
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range,history.history['loss'],label = 'Training Loss')
plt.plot(epochs_range, history.history['val_loss'],label = 'Validation Loss')
plt.title('Loss')

In [ ]:
image = "/content/plastic-bottles.jpg"
image = tf.keras.utils.load_img(image, target_size=(img_height,img_width))
img_arr = tf.keras.utils.array_to_img(image)
img_bat=tf.expand_dims(img_arr,0)

In [ ]:
predict = model.predict(img_bat)

In [ ]:
score = tf.nn.softmax(predict)

In [ ]:
print('Given image is {} with accuracy of {:0.2f}'.format(data_classes[np.argmax(score)],np.max(score)*100))

In [ ]:
model.save('Garbage_Classification.keras')

## Web Application

In [ ]:
!pip install streamlit

In [ ]:
from keras.models import  load_model
import streamlit as st


st.header('Waste Classification Model')
model = load_model('/content/Garbage_Classification.keras')


data_classes = [
    ['Cardboard',
 'Food Organics',
 'Glass',
 'Metal',
 'Miscellaneous Trash',
 'Paper',
 'Plastic',
 'Textile Trash',
 'Vegetation']
]



img_height = 180
img_width = 180
image =st.text_input('Enter Image name','plastic-bottles.jpg')

image_load = tf.keras.utils.load_img(image, target_size=(img_height,img_width))
img_arr = tf.keras.utils.array_to_img(image_load)
img_bat=tf.expand_dims(img_arr,0)



predict = model.predict(img_bat)



score = tf.nn.softmax(predict)
st.image(image, width=180)


if 0 <= np.argmax(score) < len(data_classes):

  st.write('Garbage in image is ' + data_classes[np.argmax(score)])
  st.write('With accuracy of ' + str(np.max(score)*100))

else:
  st.write('Invalid class index:', np.argmax(score))


ModuleNotFoundError: No module named 'streamlit'